<a href="https://colab.research.google.com/github/olgagasowska/Machine-Learning-for-Linguists/blob/main/Toxic_Chat_Sentiment_Classification_with_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from datasets import load_dataset
from transformers import AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

dataset = load_dataset("lmsys/toxic-chat", "toxicchat0124")

model_id = "xlm-roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)


def process(examples):
    return tokenizer(examples["user_input"], padding="max_length", truncation=True, max_length=512)
tokenized_datasets = dataset.map(process, batched=True)

filtered_train_dataset = tokenized_datasets['train'].remove_columns([col for col in tokenized_datasets['train'].column_names if col not in ["input_ids", "attention_mask", "toxicity"]])
filtered_test_dataset = tokenized_datasets['test'].remove_columns([col for col in tokenized_datasets['test'].column_names if col not in ["input_ids", "attention_mask", "toxicity"]])
#### I had no idea how to deal with it, I was trying to upload from Hugging Face, but whatever I did, I had only train/ test data,
### not both, so i asked ChatGPT and it gave me several solutions, I took this one but to be honest I could not do it myself


def convert_to_tensors(dataset, batch_size):
    print(f"First few input_ids: {dataset['input_ids'][:2]}")
    print(f"Type of input_ids: {type(dataset['input_ids'])}")

    input_ids = torch.tensor(dataset["input_ids"])
    attention_mask = torch.tensor(dataset["attention_mask"])
    labels = torch.tensor(dataset["toxicity"])

    print(f"input_ids shape: {input_ids.shape}")
    print(f"labels shape: {labels.shape}")

    return TensorDataset(input_ids, labels, attention_mask)

batch_size = 16
train_dataset = convert_to_tensors(tokenized_datasets['train'], batch_size)
test_dataset = convert_to_tensors(tokenized_datasets['test'], batch_size)


train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


class SimpleRNNClassifier(nn.Module):
    def __init__(self, vocab_size: int, embedding_dim: int, hidden_size: int, output_size: int, num_layers: int = 3):
        super(SimpleRNNClassifier, self).__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        self.rnn = nn.RNN(embedding_dim, hidden_size, num_layers=num_layers, batch_first=True, dropout=0.3, bidirectional=True)

        self.fc = nn.Linear(hidden_size * 2, output_size)

    def forward(self, x):
        embedded = self.embedding(x)

        rnn_out, hidden = self.rnn(embedded)

        out = self.fc(torch.cat((hidden[-2], hidden[-1]), dim=1))
        return out


vocab_size = tokenizer.vocab_size
embedding_dim = 128
hidden_size = 256
output_size = 2

model = SimpleRNNClassifier(vocab_size=vocab_size, embedding_dim=embedding_dim, hidden_size=hidden_size, output_size=output_size)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # I changed here, before it looked like the model was not learning at all, it was giving me the same results in each epoch
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

def train(model, dataloader, criterion, optimizer, scheduler, device):
    model.train()
    running_loss = 0.0
    for inputs, labels, attention_mask in dataloader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        attention_mask = attention_mask.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5)

        optimizer.step()

        running_loss += loss.item()
    scheduler.step()
    return running_loss / len(dataloader)

def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    failed_cases = []
    with torch.no_grad():
        for inputs, labels, attention_mask in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            attention_mask = attention_mask.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for idx, prediction in enumerate(predicted):
                if prediction != labels[idx]:
                    failed_cases.append((inputs[idx], labels[idx], prediction))

    accuracy = correct / total
    failed_count = len(failed_cases)

    print(f"Total Examples: {total}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Failed Cases: {failed_count}/{total}")

    for i, (input_tensor, true_label, predicted_label) in enumerate(failed_cases[:10]):
        print(f"Example {i+1}:")
        print(f" - Input: {tokenizer.decode(input_tensor.cpu().numpy(), skip_special_tokens=True)}")
        print(f" - True Label: {true_label.item()}, Predicted: {predicted_label.item()}")

    return accuracy, failed_count


epochs = 1000
for epoch in range(epochs):

    train_loss = train(model, train_loader, criterion, optimizer, scheduler, device)

    test_accuracy, failed_count = evaluate(model, test_loader, device)

    print(f"Epoch [{epoch+1}/{epochs}]")
    print(f"Train Loss: {train_loss:.4f}")
    print(f"Test Accuracy: {test_accuracy:.4f}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.46k [00:00<?, ?B/s]

(…)ata/0124/toxic-chat_annotation_train.csv:   0%|          | 0.00/8.20M [00:00<?, ?B/s]

data/0124/toxic-chat_annotation_test.csv:   0%|          | 0.00/8.09M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/5082 [00:00<?, ? examples/s]

Map:   0%|          | 0/5083 [00:00<?, ? examples/s]

First few input_ids: [[0, 146188, 17943, 834, 5707, 552, 36, 14, 32684, 978, 236, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

KeyboardInterrupt: 